See how to use Shor's algorithm to factor 15 here:    <b><a href="https://portal.quantumrings.com/doc/Shors.html">Shor15</a></b>

<i><b>Source code to factorize 15</b></i>

Note: Be sure to use your API token and your account name.

Step 1. Import the required modules and obtain the backend

In [1]:
# pip install QuantumRingsLib

Note: you may need to restart the kernel to use updated packages.


In [1]:
import QuantumRingsLib
from QuantumRingsLib import QuantumRegister, AncillaRegister, ClassicalRegister, QuantumCircuit
from QuantumRingsLib import QuantumRingsProvider
from QuantumRingsLib import job_monitor
from QuantumRingsLib import JobStatus
from matplotlib import pyplot as plt
import numpy as np
import math

provider = QuantumRingsProvider(
    token='rings-128.YXZRFUMMfK0iaD7TlGMbb9Z2qUh3XrBC',
    name='aryan.malhotra@rutgers.edu'
)
backend = provider.get_backend("scarlet_quantum_rings")
shots = 2048

provider.active_account()

{'name': 'aryan.malhotra@rutgers.edu',
 'token': 'rings-128.YXZRFUMMfK0iaD7TlGMbb9Z2qUh3XrBC',
 'max_qubits': '128'}

Step 2. Define the core methods

In [2]:
def gcd(a, b):
    """Compute the greatest common divisor (GCD) using Euclidean algorithm."""
    while b:
        a, b = b, a % b
    return a
def set_reg(qc, input, q, n):
    """
    Sets a quantum register with an input value.
    Args:
        qc (QuantumCircuit): The quantum circuit to use
        input (int): The number to be stored in the qubit register
        q (QuantumRegister): The qubit register which is to be programmed with the input number
        n (int): The width of the qubit register to use..
    """
    for i in range (0, n):
        if ((1 << i) & input ):
            qc.x(q[i])
    return
    
def iqft_cct(qc, b, n):
    """
    The inverse QFT circuit
    Args:
        qc (QuantumCircuit):The quantum circuit
        b (QuantumRegister):The target register
        n (int):The number of qubits in the registers to use
    Returns:
        None
    """
    for i in range (n):
        for j in range (1, i+1):
            # for inverse transform, we have to use negative angles
            qc.cu1(  -np.pi / 2** ( i -j + 1 ), b[j - 1], b[i])
        # the H transform should be done after the rotations
        qc.h(b[i])
    qc.barrier()
    return

def mult_cct(qc, input1, input2, a, b, s, n):
    """
    a*b multiplier circuit using QFT
    Args:
        qc (QuantumCircuit):The quantum circuit
        input1 (int):The multiplicand
        input2 (int):The multiplier
        s (QuantumRegister):The qubit register holding product a * b
            Note: s has two times the length of a and b
        a (QuantumRegister):The qubit register for multiplicand
        b (QuantumRegister):The qubit register for multiplier
        n (int):The length of the registers
    Returns:
        None.
    """
    set_reg(qc, input1, a, n)
    set_reg(qc, input2, b, n)
    for i in range (0, n):
        c_add_qft(qc, b, s, i,  a[i], n)
    return

def c_add_qft(qc, a, b, offset, control, n):
    """
        The controlled Adder using QFT.
        Args:
            qc (QuantumCircuit):The quantum circuit
            a (QuantumRegister):The source register
            b (QuantumRegister):The target register.
            offset (int):The starting qubit in register b to work from
            control (int):The index number of the control qubit
            n (int):The number of qubits in the registers to use
        Returns: None
    """
    c_qft_cct(qc, b, offset, control, n)
    c_add_cct(qc, a, b, offset, control, n)
    c_iqft_cct(qc, b, offset, control, n)
    return

def c_add_cct(qc, a, b, offset, control, n):
    """
        The controlled adder circuit module.
        Args:
            qc (QuantumCircuit):The quantum circuit
            a (QuantumRegister):The source register
            b (QuantumRegister):The target register. Computed value is stored in this register
            offset (int):The starting qubit in register b to work from
            n (int):The number of qubits in the registers to use
        Returns:
            None
    """
    while (n):
        for i in range(n, 0, -1):
            ccu1(qc, math.pi/2**(n - i), control, a[i-1], b[n-1+offset])
        qc.barrier()
        n -= 1
    return



def c_iqft_cct (qc, b, offset, control, n):
    """
        The controlled Inverse-QFT.
        Args:
            qc (QuantumCircuit):The quantum circuit
            b (QuantumRegister):The target register.
            offset (int):The starting qubit in register b to work from
            control (int):The index number of the control qubit
            n (int):The number of qubits in the registers to use
        Returns:
            None
    """
    for i in range (n):
        for j in range (1, i+1):
            # for inverse transform, we have to use negative angles
            ccu1( qc, -math.pi / 2** ( i -j + 1 ), control, b[j - 1+offset], b[i+offset])
        # the H transform should be done after the rotations
        qc.ch(control, b[i+offset])
    qc.barrier()
    return

def c_qft_cct(qc, b, offset, control, n):
    """
        The controlled QFT.
        Args:
            qc (QuantumCircuit:The quantum circuit
            b (QuantumRegister):The target register.
            offset (int):The starting qubit in register b to work from
            control (int):The index number of the control qubit
            n (int):The number of qubits in the registers to use
        Returns:
            None
    """
    while (n):
        qc.ch(control, b[n + offset - 1])
        for i in range (n-1, 0, -1):
            ccu1(qc, math.pi / 2** (n - i), control, b[i - 1 + offset], b[n + offset - 1])
        n -= 1
    qc.barrier()

    return
def ccu1(qc, theta, q0, q1, q2):
    """
    The controlled Adder using QFT.
    Args:
        qc (QuantumCircuit):The quantum circuit
        theta (float):The rotational angle. See :ref:`QuantumCircuit.u1`
        q0 (int):The first control qubit.
        q1 (int):The second control qubit.
        q2 (int):The target qubit.
    Returns:None
    """
    qc.cu1(theta/2, q1, q2)
    qc.cx(q0, q1)
    qc.cu1(-theta/2, q1, q2)
    qc.cx(q0, q1)
    qc.cu1(theta/2, q0, q2)
    return
    
def plot_histogram (counts, title=""):
    """
    Plots the histogram of the counts
    Args:
        counts (dict):The dictionary containing the counts of states
        titles (str):A title for the graph.
    Returns:None
    """
    fig, ax = plt.subplots(figsize =(10, 7))
    plt.xlabel("States")
    plt.ylabel("Counts")
    mylist = [key for key, val in counts.items() for _ in range(val)]

    unique, inverse = np.unique(mylist, return_inverse=True)
    bin_counts = np.bincount(inverse)

    plt.bar(unique, bin_counts)

    maxFreq = max(counts.values())
    plt.ylim(ymax=np.ceil(maxFreq / 10) * 10 if maxFreq % 10 else maxFreq + 10)
    # Show plot
    plt.title(title)
    plt.show()
    return

def test_mult(value, exponent):
    numberofqubits = max(value, exponent, 4) #Might replace exp with value
    a = QuantumRegister(numberofqubits , 'a')
    b = QuantumRegister(numberofqubits , 'b')
    s = QuantumRegister(numberofqubits*2 , 's')
    c = ClassicalRegister(numberofqubits*2 , 'c')
    qc = QuantumCircuit(a, b, s, c)
    mult_cct(qc, value, exponent, a, b, s, numberofqubits)
    for i in range (numberofqubits*2):
        qc.measure(s[i],c[i])
    # Execute the circuit
    job = backend.run(qc, shots=shots)
    job_monitor(job)
    result = job.result()
    counts = result.get_counts()
    
    if (1 < len(counts)):
        print("Error: More than one result!")
        myproduct = 0
    else:
        scounts = str(counts)
        myproduct = int("0b"+scounts[scounts.index("{")+2:scounts.index(":")-1], 2)
    return myproduct

Step 3. Perform the algorithm

In [ ]:
# Shor’s algorithm to factorize 15 using 7^x mod 15.
numberofqubits = 20
shots = 1024*16
N = 15
x0 = 3
x = 7
r = 0
print("Starting value: ",x)
while x != x0 or r == 0:
    r = r+1
    res = test_mult(x, x0)
    print(res)
    x = res %N
    print(x)
    
print ("Result is: ", r)

# Draw the circuit
#qc.draw('mpl') Very large circuit. Do not recommend drawing

Starting value:  3
Job Running
Job Done.
Ending Job Monitor
9
9
Job Queued
Job Running
Job Running
Job Done.
Ending Job Monitor
27
12
Job Queued
Job Running
Job Running
Job Running
Job Running
Job Done.
Ending Job Monitor
36
6
Job Queued
Job Running
Job Done.
Ending Job Monitor
18
3
Result is:  4


The circuit to factor 15 shown above.

In [6]:
# visualize
#print (counts)
#plot_histogram(counts)

p = gcd(x0**(r/2) + 1,N) 
q = gcd(x0**(r/2) - 1,N)
print ("Result is: ", r)
print("The prime factors are:",p,q)
#clean up


Result is:  4
The prime factors are: 5.0 1.0


A plot of the execution results is shown above. Compare this with the calculated values.

Footnotes

[1] This section is based on [10], [14], and [16].

[2] https://research.ibm.com/blog/factor-15-shors-algorithm

[3] https://en.wikipedia.org/wiki/Integer_factorization_records#Records_for_efforts_by_quantum_computers

In [6]:
#Post Processing


In [11]:
gcd(-4,10)

2